<a href="https://colab.research.google.com/github/amilkh/cs230-fer/blob/master/datasets/affectnet_to_png.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
! rm -rf /content/affectnet && mkdir /content/affectnet
! unrar e -idq '/content/drive/My Drive/cs230 project/dataset/affectnet/Manually_Annotated.part01.rar' /content/affectnet


Cannot find volume /content/drive/My Drive/cs230 project/dataset/affectnet/Manually_Annotated.part06.rar
Manually_Annotated_Images/461/d45ace15e1ed59b21e7665fe337d056d7236bf1b668a0b64cdaeea12.jpg - checksum error

In [6]:
! ls -1 /content/affectnet | wc -l

193591


In [0]:
import pandas as pd

train_csv = pd.read_csv('/content/drive/My Drive/cs230 project/dataset/affectnet/training.csv')

# FER2013 = {0:'angry', 1:'disgust', 2:'fear', 3:'happy', 4:'sad', 5:'surprise', 6:'neutral'}
# AffectNet = 0: Neutral, 1: Happiness, 2: Sadness, 3: Surprise, 4: Fear, 5: Disgust, 6: Anger, 7: Contempt, 8: None, 9: Uncertain, 10: No-Face
def reindex_labels(y):
    label_mapping = {0:6,1:3,2:4,3:5,4:2,5:1,6:0,7:7,8:8,9:9,10:10}
    for label in label_mapping.keys():
        y[y == label] = label_mapping[label]

    return y

In [7]:
train_csv[train_csv['subDirectory_filePath']=='163/7b5d3644888582a32c7896e7f58f400f179f1fd8c48feb286e2cd8d4.jpg']

,subDirectory_filePath,face_x,face_y,face_width,face_height,facial_landmarks,expression,valence,arousal
11,163/7b5d3644888582a32c7896e7f58f400f179f1fd8c4...,100,100,669,669,220.53;366.29;223.18;430.64;234.1;493.62;250.6...,10,-2.0,-2.0


In [0]:
train_csv['subDirectory_filePath'] = train_csv['subDirectory_filePath'].apply(lambda fp: [fp.split('/')[1]])

In [12]:
train_csv(0)

In [0]:
from PIL import Image

for i, row in train_csv.iterrows():
  try:
    fn=row['subDirectory_filePath'].split('/')[1][0]
    img=Image.open('/content/affectnet/'+fn)
  except IOError as e:
    pass
  else:
    img.show()
    break

In [0]:
train_csv[train_csv['subDirectory_filePath'] == train_csv.iloc[0]['subDirectory_filePath'][0]]['subDirectory_filePath']

Series([], Name: subDirectory_filePath, dtype: object)

In [0]:
from os import listdir
for fn in listdir('/content/affectnet'):
    if fn.endswith(".jpg"):
      print(train_csv['subDirectory_filePath'==fn])
      break
    else:
      continue

In [0]:
#e = 0
#train_csv_e=train_csv[train_csv['expression']==e]



(74874, 9)

In [0]:
import numpy as np

X = np.load('/content/ck-output/npy_files/X.npy')
y8 = np.load('/content/ck-output/npy_files/y.npy')
y = reindex_labels(y8)

In [0]:
import matplotlib.pyplot as plt

emotions = {0:'angry', 1:'disgust', 2:'fear', 3:'happy', 4:'sad', 5:'surprise', 6:'neutral'}

for i in range(0,10):
  plt.xlabel(emotions[y[i]])
  plt.imshow(X[i].reshape(96,96),cmap='gray')
  plt.figure()

In [0]:
X.shape

(1232, 1, 96, 96)

In [0]:
from os import mkdir
mkdir('/content/ck-images/')
for emotion in emotions:
  mkdir(f'/content/ck-images/' + f'{emotion} ' + f'{emotions[emotion]}')

In [0]:
from PIL import Image

count = 0
for i in range(0,X.shape[0]):
    count_string = str(count).zfill(7)
    fname = '/content/ck-images/' + f'{y[i]} ' + f'{emotions[y[i]]}/' + f'{emotions[y[i]]}-{count_string}.png'
    img = Image.fromarray(X[i].reshape(96,96))
    img=img.resize((48,48))
    img.save(fname) 
    count += 1

In [0]:
! cd ck-images && zip -r ck-plus.zip *

In [0]:
! mv ck-images/ck-plus.zip '/content/drive/My Drive/cs230 project/dataset/'

In [0]:
! ls -lh '/content/drive/My Drive/cs230 project/dataset/'

total 218M
drwx------  2 root root 4.0K Mar 15 04:00  ck+
-rw-------+ 1 root root 2.3M Mar 18 19:49  ck-plus.zip
drwx------  2 root root 4.0K Mar 15 02:57  emotion
-rw-------  1 root root  50M Mar 15 03:01  emotion.zip
-rw-------  1 root root  51M Mar 15 18:29  facesdb.zip
drwx------  2 root root 4.0K Mar 12 17:11  fer2013
-rw-------  1 root root 299K Mar 15 15:53  googlesearch.zip
drwx------  2 root root 4.0K Mar 15 03:15  googleset
-rw-------  1 root root 3.8M Mar 15 03:15  googleset.zip
drwx------  2 root root 4.0K Mar 12 21:21  jaffedbase
-rw-------  1 root root  12M Mar 15 18:29  jaffe.zip
drwx------  2 root root 4.0K Mar 15 15:32  old
-rw-------  1 root root 100M Mar 15 15:32  umea.zip
dr-x------  2 root root 4.0K Mar 12 17:11  webcam
-rw-------  1 root root 588K Mar 18 00:04 'webcam amil.zip'
-rw-------  1 root root 346K Mar 17 23:46 'webcam paki.zip'
